## Strategy based on slope of Close time series

This signal is based on the slope of tickers close price time series over the 5 last trading days
Close price are previously smoothed with a exponential mean


The signal is magnified by taking the maximum of the 21 days last slope values, multiplied by the sign of the slope time series last data point

In [ ]:
import pandas as pd
import numpy as np
from data_master import DataMaster
from utils import func
from scipy.stats import norm
master = DataMaster()
from utils.func import center
from quantstats.stats import sharpe
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
market_data = pd.read_parquet('data/US/mkt_data.pq')
balance_sheet = pd.read_parquet('data/US/balance_sheets.pq')
GICS = pd.read_parquet('data/US/GICS.pq')
P = pd.read_parquet('data/US/universe_table.pq')
R = market_data['close'].unstack().reindex_like(P).pct_change()
market_data.dropna().sample(5)

In [ ]:

def slope_price( close, length=None, as_angle=None, to_degrees=None, vertical=None, offset=None, **kwargs):

    length = int(length) if length and length > 0 else 1
    as_angle = True if isinstance(as_angle, bool) else False
    to_degrees = True if isinstance(to_degrees, bool) else False
    offset   = int(offset) if offset else 0

    if close is None: return

    slope = close.diff(length) / length
    if as_angle:
        slope = slope.apply(np.arctan)
        if to_degrees:
            slope *= 180 / np.pi

    if offset != 0:
        slope = slope.shift(offset)


    if "fillna" in kwargs:
        slope.fillna(kwargs["fillna"], inplace=True)
    if "fill_method" in kwargs:
        slope.fillna(method=kwargs["fill_method"], inplace=True)


    return slope

In [ ]:
results ={}
for ticker in tqdm(market_data.index.get_level_values(1).unique()):
    df_ticker = market_data.xs(ticker,level=1)
    try :
        metrics = slope_price(df_ticker['close'].ewm(alpha=0.5).mean(),length=5)
        results[ticker] = metrics
    except :
        print(ticker)

In [ ]:
signal = pd.concat(results,axis=1).reindex_like(P).ffill()
signal = signal[P].abs().rolling(21).max() * np.sign(signal[P])
signal = signal[P].rank(axis=1,pct=True,ascending=False).clip(0.01,0.99).apply(norm.ppf)
signal[signal.abs()<0.5] = None

signal = signal[P].groupby(GICS['gicgrp'],axis=1).apply(center)
signal = signal[P].div(signal[P].abs().sum(1),0)

PNL = (signal[P].shift()*R).sum(1)
sr = sharpe(PNL)
turnover = signal.fillna(0).diff().abs().sum(1).mean()
PNL.cumsum().plot()
pd.Series({'Sharpe':sr,'Average Daily turnover (%)':turnover *100,'Average daily pnl (bps) ':PNL.mean()*1e4}).round(1)

In [ ]:
signal.count(1).plot(label ='# positions',legend=True)

In [ ]:
signal.abs().sum(1).plot(label = 'absolute weights sum',legend=True)
signal.sum(1).plot(label='weights sum',legend=True)

In [ ]:
import quantstats as qs
SP500 = qs.utils.download_returns('SPY')

# report with fancy (and some unuseful) stats
qs.reports.html(PNL, "SPY",title = 'Strategy Slope')

In [ ]:
PNL.to_csv('data/US/strat_slope_pnl.csv')

In [9]:
pip install learning2rank

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement learning2rank (from versions: none)
ERROR: No matching distribution found for learning2rank
You should consider upgrading via the 'c:\Users\marti\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.
